Importing Libraries and Loading the Data:


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Conv1D, MaxPooling1D, Dense, Dropout, SpatialDropout1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import nltk
from nltk.corpus import stopwords
import string

# Download stopwords
nltk.download('stopwords')

# Load the dataset
file_path = '/content/HateSpeechDetection (Balanced dataset).csv'  # Update the file path
df = pd.read_csv(file_path)

# Display the first few rows of the dataframe
print(df.head())


  Platform                                            Comment  Hateful
0   Reddit  Damn I thought they had strict gun laws in Ger...        0
1   Reddit  I dont care about what it stands for or anythi...        0
2   Reddit                  It's not a group it's an idea lol        0
3   Reddit                          So it's not just America!        0
4   Reddit  The dog is a spectacular dancer considering he...        0


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Data Preprocessing:

In [3]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

# Apply the preprocessing function to the comments
df['Comment'] = df['Comment'].apply(preprocess_text)

# Display the first few rows of the preprocessed dataframe
print(df.head())


  Platform                                           Comment  Hateful
0   Reddit              damn thought strict gun laws germany        0
1   Reddit  dont care stands anything connected like shields        0
2   Reddit                                    group idea lol        0
3   Reddit                                           america        0
4   Reddit  dog spectacular dancer considering two left feet        0


Tokenization and Padding

In [4]:
# Split the data into features and labels
X = df['Comment']
y = df['Hateful']

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_tokenized = tokenizer.texts_to_sequences(X)

# Pad the sequences
max_length = 100  # Define the maximum length for padding
X_padded = pad_sequences(X_tokenized, maxlen=max_length, padding='post')

# Display the shape of the padded data
print(f'Padded data shape: {X_padded.shape}')


Padded data shape: (3000, 100)


Splitting the Data:

In [5]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

print(f'Training samples: {len(X_train)}')
print(f'Testing samples: {len(X_test)}')


Training samples: 2400
Testing samples: 600


 Building the Model:

In [6]:
# Define the model
embedding_dim = 128  # Dimension of the embedding vectors
model = Sequential()

# Add embedding layer
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim, input_length=max_length))

# Add spatial dropout layer
model.add(SpatialDropout1D(0.2))

# Add convolutional layer
model.add(Conv1D(filters=64, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

# Add LSTM layer
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))

# Add dense layers
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))  # Sigmoid for binary classification

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Display the model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          792704    
                                                                 
 spatial_dropout1d (Spatial  (None, 100, 128)          0         
 Dropout1D)                                                      
                                                                 
 conv1d (Conv1D)             (None, 96, 64)            41024     
                                                                 
 max_pooling1d (MaxPooling1  (None, 48, 64)            0         
 D)                                                              
                                                                 
 lstm (LSTM)                 (None, 100)               66000     
                                                                 
 dense (Dense)               (None, 100)               1

Training the Model:

In [8]:
# Train the model
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2, verbose=1)


Epoch 1/5
60/60 [==============================] - 14s 159ms/step - loss: 0.5213 - accuracy: 0.7948 - val_loss: 0.5078 - val_accuracy: 0.7958
Epoch 2/5
60/60 [==============================] - 8s 128ms/step - loss: 0.5161 - accuracy: 0.7937 - val_loss: 0.5062 - val_accuracy: 0.7958
Epoch 3/5
60/60 [==============================] - 9s 145ms/step - loss: 0.5143 - accuracy: 0.7937 - val_loss: 0.5077 - val_accuracy: 0.7958
Epoch 4/5
60/60 [==============================] - 9s 153ms/step - loss: 0.5190 - accuracy: 0.7937 - val_loss: 0.5073 - val_accuracy: 0.7958
Epoch 5/5
60/60 [==============================] - 7s 118ms/step - loss: 0.5130 - accuracy: 0.7937 - val_loss: 0.5075 - val_accuracy: 0.7958


Evaluating the Model:

In [7]:
# Evaluate the model
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy}')

# Print classification report
print(classification_report(y_test, y_pred))


19/19 [==============================] - 3s 55ms/step - loss: 0.6902 - accuracy: 0.8233
Test Accuracy: 0.8233333230018616
              precision    recall  f1-score   support

           0       0.82      1.00      0.90       494
           1       0.00      0.00      0.00       106

    accuracy                           0.82       600
   macro avg       0.41      0.50      0.45       600
weighted avg       0.68      0.82      0.74       600



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
